Analysis
With a total player count of 576, the population for the game is mostly male, consisiting of about 84% of the total.
With Purchase trends, the key demographics for purchases are for those in the 20-24 age range. And looking at the average purchase prices, Gender does not have a significant role for making in game purchases.
Lower priced items aren't necessarily more popular, which shows us that the items being purhcased are most likely for their attributes in game. So to speak, the better bang for your buck.



In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Create reference to CSV file
csv_path = "Resources/purchase_data.csv"

# Import the CSV into a pandas DataFrame
p_data = pd.read_csv(csv_path, low_memory=False)
p_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Get Player COunt
player_count = len(p_data['SN'].unique())
print(f"Player Count: {player_count}")

Player Count: 576


In [35]:
#Purchasing Analysis (Total)
unique_items = len(p_data['Item Name'].unique())
average_pr_price = '${:,.2f}'.format(p_data['Price'].mean())
purchase_count = len(p_data)
total_revenue = '${:,.2f}'.format(p_data['Price'].sum())


print(f"Unique Items: {unique_items}")
print(f"Average Purchase Price: {average_pr_price}")
print(f"Total Purchases: {purchase_count}")
print(f"Total Revenue: {total_revenue}")

Unique Items: 179
Average Purchase Price: $3.05
Total Purchases: 780
Total Revenue: $2,379.77


In [45]:
#Gender Demographics

grouped= p_data.groupby('Gender').SN.nunique()
grouped.head()
grouped= pd.DataFrame(grouped)
grouped = grouped.rename(columns={'SN': 'Count'})
grouped['Percentage'] = grouped['Count'].astype('float')/grouped['Count'].sum()*100
grouped['Percentage'] = grouped['Percentage'].map("{:.2f}%".format)
grouped

,Count,Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [51]:
#Purchasing Analysis
pA = p_data.groupby("Gender").agg({"Price": np.sum, "SN": pd.Series.nunique})
pA = pA.rename(columns={"SN": 'Purchase Count', "Price": "Total Purchase Value"})
pA['Average Purchase Price']= pA['Total Purchase Value']/pA['Purchase Count']

pA['Average Purchase Price']= pA['Average Purchase Price'].map("${:.2f}".format)
pA['Total Purchase Value']=pA['Total Purchase Value'].map("${:.2f}".format)

pA

,Total Purchase Value,Purchase Count,Average Purchase Price
Gender,,,
Female,$361.94,81,$4.47
Male,$1967.64,484,$4.07
Other / Non-Disclosed,$50.19,11,$4.56


In [36]:
# Create bins 
bins=[0, 9, 14,19,24,29,34,39,100]
# Create labels for these bins
gnames=['10 Below','10 - 14', '15-19','20-24','25-29','30-34','35-39','40 and Above']
pd.cut(p_data["Age"], bins, labels=gnames).head()

# Create a GroupBy object based upon "View Group"
aged=p_data.groupby("Age")
# Slice the data and place it into bins
p_data['Age Group'] = pd.cut(p_data["Age"], bins, labels=gnames)
p_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40 and Above
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [47]:
ageGroup= p_data.groupby('Age Group')

groupings = pd.DataFrame(p_data['Age Group'].value_counts())
groupings = groupings.rename(columns={"Age Group": 'Purchase Count'})
groupings['Total Purchase Value'] = ageGroup['Price'].sum()
groupings['Average Purchase Price'] = ageGroup['Price'].sum()/groupings['Purchase Count']

groupings['Total Purchase Value']=groupings['Total Purchase Value'].map("${:.2f}".format)
groupings['Average Purchase Price']=groupings['Average Purchase Price'].map("${:.2f}".format)
groupings

,Purchase Count,Total Purchase Value,Average Purchase Price
20-24,365,$1114.06,$3.05
15-19,136,$412.89,$3.04
25-29,101,$293.00,$2.90
30-34,73,$214.00,$2.93
35-39,41,$147.67,$3.60
10 - 14,28,$82.78,$2.96
10 Below,23,$77.13,$3.35
40 and Above,13,$38.24,$2.94


In [48]:
#TOP 5 SPENDERS

spenders_group= p_data.groupby('SN')
spenders_group_df=spenders_group.count()
spenders_group_df.drop(columns={'Purchase ID','Age','Age Group','Item ID','Price', 'Item Name'}, inplace=True)
spenders_group_df.rename(columns={'Gender': 'Purchase Count'}, inplace= True)

spenders_group_df['Total Purchase Value'] =spenders_group['Price'].sum()
spenders_group_df['Average Purchase Price']=spenders_group_df['Total Purchase Value']/spenders_group_df['Purchase Count']

spenders_group_df['Total Purchase Value']=spenders_group_df['Total Purchase Value'].map("${:.2f}".format)
spenders_group_df['Average Purchase Price']=spenders_group_df['Average Purchase Price'].map("${:.2f}".format)

spenders_group_df.sort_values(by=['Purchase Count'], ascending=False).head(5)

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Iral74,4,$13.62,$3.40
Idastidru52,4,$15.45,$3.86
Asur53,3,$7.44,$2.48
Inguron55,3,$11.11,$3.70


In [49]:
#Most Popular Items
pop_items= p_data.groupby('Item Name')
# pop_count =pop_items.count().sort_values(by="Item ID", ascending=False).head(5)
pop_count =pop_items.count()

pop_sum =pop_items.sum()
pop_merge = pd.merge(pop_sum, pop_count, on="Item Name")
pop_merge.drop(columns={'Purchase ID_x','Age_x','Purchase ID_y','SN','Age_y','Gender','Item ID_y','Price_y'}, inplace=True)
pop_merge.rename(columns={"Item ID_x": 'Item ID', 'Price_x': 'Total Purchase Value','Age Group': 'Purchase Count'}, inplace=True)

pop_merge['Item ID']=(pop_merge['Item ID']/pop_merge['Purchase Count']).astype('int')
pop_merge['Item Price']=pop_merge['Total Purchase Value']/pop_merge['Purchase Count']

pop_merge['Total Purchase Value']=pop_merge['Total Purchase Value'].map("${:.2f}".format)
pop_merge['Item Price']=pop_merge['Item Price'].map("${:.2f}".format)

pop_merge.sort_values(by=['Purchase Count'], ascending=False).head(5)

,Item ID,Total Purchase Value,Purchase Count,Item Price
Item Name,,,,
Final Critic,95,$59.99,13,$4.61
"Oathbreaker, Last Hope of the Breaking Storm",178,$50.76,12,$4.23
Persuasion,139,$28.99,9,$3.22
Nirvana,82,$44.10,9,$4.90
"Extraction, Quickblade Of Trembling Hands",108,$31.77,9,$3.53


In [50]:
#Most Profitable items
pop_merge.sort_values(by=['Item Price'], ascending=False).head(5)

,Item ID,Total Purchase Value,Purchase Count,Item Price
Item Name,,,,
Stormfury Mace,63,$9.98,2,$4.99
"Mercy, Katana of Dismay",139,$24.70,5,$4.94
"Hellreaver, Heirloom of Inception",147,$14.79,3,$4.93
Stormfury Longsword,173,$9.86,2,$4.93
"Blazeguard, Reach of Eternity",128,$24.55,5,$4.91
